# 安徽安庆市项目`WRF-CMAQ`模拟分析
## 模拟结果预处理：*`CHEM-Data`*

---
*@author: Evan*\
*@date: 2023-06-06*

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from functools import reduce

from matplotlib import rcParams
config = {
    "font.family":'Times New Roman',
    "mathtext.fontset":'stix',
    "font.serif": ['SimSun'],
}
rcParams.update(config)

import cartopy.crs as ccrs
import cartopy.feature as cfeat
from cartopy.io.shapereader import Reader

# silence the warning note
import warnings
warnings.filterwarnings("ignore")

In [ ]:
pa = xr.open_dataset('F:/Data/Project_anqing/May/COMBINE_ACONC_v54_ISAM_intel_CN3AH_135X138_2023_pa.nc')
grid = xr.open_dataset('F:/Data/Project_anqing/GRIDCRO2D_2023021.nc')

In [2]:
files = ['F:/Data/Project_anqing/May/COMBINE_pa_15.nc',
         'F:/Data/Project_anqing/May/COMBINE_pa_16-31.nc']
pa = xr.open_mfdataset(files,combine='nested',concat_dim='TSTEP')
grid = xr.open_dataset('F:/Data/Project_anqing/GRIDCRO2D_2023021.nc')

In [3]:
# 将时间点转换为实际时间
times=pd.date_range('2023-05-16-00','2023-05-31-23',freq='h')
times

DatetimeIndex(['2023-05-16 00:00:00', '2023-05-16 01:00:00',
               '2023-05-16 02:00:00', '2023-05-16 03:00:00',
               '2023-05-16 04:00:00', '2023-05-16 05:00:00',
               '2023-05-16 06:00:00', '2023-05-16 07:00:00',
               '2023-05-16 08:00:00', '2023-05-16 09:00:00',
               ...
               '2023-05-31 14:00:00', '2023-05-31 15:00:00',
               '2023-05-31 16:00:00', '2023-05-31 17:00:00',
               '2023-05-31 18:00:00', '2023-05-31 19:00:00',
               '2023-05-31 20:00:00', '2023-05-31 21:00:00',
               '2023-05-31 22:00:00', '2023-05-31 23:00:00'],
              dtype='datetime64[ns]', length=384, freq='H')

In [4]:
# 将层数转换为气压高度
preslevel=np.array(
    [1.,     0.9979, 0.9956, 0.9931, 0.9904, 0.9875, 0.9844, 0.9807, 0.9763, 0.9711,
     0.9649, 0.9575, 0.9488, 0.9385, 0.9263, 0.912,  0.8951, 0.8753, 0.8521, 0.8251,
     0.7937, 0.7597, 0.7229, 0.6883, 0.641,  0.596,  0.5484, 0.4985, 0.4467, 0.3934,
     0.3393, 0.285,  0.2316, 0.1801, 0.1324, 0.0903, 0.0542, 0.0241,]
    )
pres=preslevel*950+50
pres

array([1000.   ,  998.005,  995.82 ,  993.445,  990.88 ,  988.125,
        985.18 ,  981.665,  977.485,  972.545,  966.655,  959.625,
        951.36 ,  941.575,  929.985,  916.4  ,  900.345,  881.535,
        859.495,  833.845,  804.015,  771.715,  736.755,  703.885,
        658.95 ,  616.2  ,  570.98 ,  523.575,  474.365,  423.73 ,
        372.335,  320.75 ,  270.02 ,  221.095,  175.78 ,  135.785,
        101.49 ,   72.895])

In [5]:
days=1 # set spin-up days
dataset1=xr.Dataset(
    data_vars=dict(
        # ! vars from PA
        HADV_O3=(['time','level','y','x'],pa.HADV_O3[days*24-8:-8,:,:,:].data*48/22.4,{'long name':'Horizontal Advection of O3','units':'ug m-3'}),
        ZADV_O3=(['time','level','y','x'],pa.ZADV_O3[days*24-8:-8,:,:,:].data*48/22.4,{'long name':'Veritical Advection of O3','units':'ug m-3'}),
        HDIF_O3=(['time','level','y','x'],pa.HDIF_O3[days*24-8:-8,:,:,:].data*48/22.4,{'long name':'Horizontal Diffusion of O3','units':'ug m-3'}),
        VDIF_O3=(['time','level','y','x'],pa.VDIF_O3[days*24-8:-8,:,:,:].data*48/22.4,{'long name':'Vertical Diffusion of O3','units':'ug m-3'}),
        DDEP_O3=(['time','level','y','x'],pa.DDEP_O3[days*24-8:-8,:,:,:].data*48/22.4,{'long name':'Dry Deposition of O3','units':'ug m-3'}),
        CHEM_O3=(['time','level','y','x'],pa.CHEM_O3[days*24-8:-8,:,:,:].data*48/22.4,{'long name':'Gas-phase Chemistry of O3','units':'ug m-3'}),

        HNO3prod=(['time','level','y','x'],pa.HNO3prod[days*24-8:-8,:,:,:].data,{'long name':'Production Velocity of HNO3','units':'ppbv'}),
        H2O2prod=(['time','level','y','x'],pa.H2O2prod[days*24-8:-8,:,:,:].data,{'long name':'Production Velocity of H2O2','units':'ppbv'}),
        O3prod=(['time','level','y','x'],pa.O3prod[days*24-8:-8,:,:,:].data*48/22.4,{'long name':'Production Velocity of O3','units':'ug m-3'}),
        O3loss=(['time','level','y','x'],pa.O3loss[days*24-8:-8,:,:,:].data*48/22.4,{'long name':'Loss Velocity of O3','units':'ug m-3'}),
        ),
    coords=dict(
        time=times,
        level=pres,
        latitude=(['y','x'],grid.LAT[0,0,:,:].data),
        longitude=(['y','x'],grid.LON[0,0,:,:].data),
    ),
    attrs=dict(
        case='Anqing_202305',
        grid='CN3AH_135X138',
    ),
)
dataset1

<xarray.Dataset>
Dimensions:    (time: 384, level: 38, y: 138, x: 135)
Coordinates:
  * time       (time) datetime64[ns] 2023-05-16 ... 2023-05-31T23:00:00
  * level      (level) float64 1e+03 998.0 995.8 993.4 ... 135.8 101.5 72.89
    latitude   (y, x) float32 29.01 29.01 29.01 29.01 ... 32.7 32.7 32.7 32.7
    longitude  (y, x) float32 114.7 114.7 114.7 114.8 ... 119.0 119.0 119.1
Dimensions without coordinates: y, x
Data variables:
    HADV_O3    (time, level, y, x) float32 dask.array<chunksize=(8, 38, 138, 135), meta=np.ndarray>
    ZADV_O3    (time, level, y, x) float32 dask.array<chunksize=(8, 38, 138, 135), meta=np.ndarray>
    HDIF_O3    (time, level, y, x) float32 dask.array<chunksize=(8, 38, 138, 135), meta=np.ndarray>
    VDIF_O3    (time, level, y, x) float32 dask.array<chunksize=(8, 38, 138, 135), meta=np.ndarray>
    DDEP_O3    (time, level, y, x) float32 dask.array<chunksize=(8, 38, 138, 135), meta=np.ndarray>
    CHEM_O3    (time, level, y, x) float32 dask.array<chunksize=(8, 38, 138, 135), meta=np.ndarray>
    HNO3prod   (time, level, y, x) float32 dask.array<chunksize=(8, 38, 138, 135), meta=np.ndarray>
    H2O2prod   (time, level, y, x) float32 dask.array<chunksize=(8, 38, 138, 135), meta=np.ndarray>
    O3prod     (time, level, y, x) float32 dask.array<chunksize=(8, 38, 138, 135), meta=np.ndarray>
    O3loss     (time, level, y, x) float32 dask.array<chunksize=(8, 38, 138, 135), meta=np.ndarray>
Attributes:
    case:     Anqing_202305
    grid:     CN3AH_135X138

In [6]:
compression=dict(zlib=True,complevel=5)
encoding={var:compression for var in dataset1.data_vars}
dataset1.to_netcdf('D:/Download/May_PA_O3-2.nc',encoding=encoding)

In [5]:
days=1 # set spin-up days
dataset2=xr.Dataset(
    data_vars=dict(
        # ! vars from PA
        HADV_NO=(['time','level','y','x'],pa.HADV_NO[days*24-8:-8,:,:,:].data*31/22.4,{'long name':'Horizontal Advection of NO','units':'ug m-3'}),
        ZADV_NO=(['time','level','y','x'],pa.ZADV_NO[days*24-8:-8,:,:,:].data*31/22.4,{'long name':'Veritical Advection of NO','units':'ug m-3'}),
        HDIF_NO=(['time','level','y','x'],pa.HDIF_NO[days*24-8:-8,:,:,:].data*31/22.4,{'long name':'Horizontal Diffusion of NO','units':'ug m-3'}),
        VDIF_NO=(['time','level','y','x'],pa.VDIF_NO[days*24-8:-8,:,:,:].data*31/22.4,{'long name':'Vertical Diffusion of NO','units':'ug m-3'}),
        DDEP_NO=(['time','level','y','x'],pa.DDEP_NO[days*24-8:-8,:,:,:].data*31/22.4,{'long name':'Dry Deposition of NO','units':'ug m-3'}),
        CHEM_NO=(['time','level','y','x'],pa.CHEM_NO[days*24-8:-8,:,:,:].data*31/22.4,{'long name':'Gas-phase Chemistry of NO','units':'ug m-3'}),
        
        HADV_NO2=(['time','level','y','x'],pa.HADV_NO2[days*24-8:-8,:,:,:].data*47/22.4,{'long name':'Horizontal Advection of NO2','units':'ug m-3'}),
        ZADV_NO2=(['time','level','y','x'],pa.ZADV_NO2[days*24-8:-8,:,:,:].data*47/22.4,{'long name':'Veritical Advection of NO2','units':'ug m-3'}),
        HDIF_NO2=(['time','level','y','x'],pa.HDIF_NO2[days*24-8:-8,:,:,:].data*47/22.4,{'long name':'Horizontal Diffusion of NO2','units':'ug m-3'}),
        VDIF_NO2=(['time','level','y','x'],pa.VDIF_NO2[days*24-8:-8,:,:,:].data*47/22.4,{'long name':'Vertical Diffusion of NO2','units':'ug m-3'}),
        DDEP_NO2=(['time','level','y','x'],pa.DDEP_NO2[days*24-8:-8,:,:,:].data*47/22.4,{'long name':'Dry Deposition of NO2','units':'ug m-3'}),
        CHEM_NO2=(['time','level','y','x'],pa.CHEM_NO2[days*24-8:-8,:,:,:].data*47/22.4,{'long name':'Gas-phase Chemistry of NO2','units':'ug m-3'}),
        ),
    coords=dict(
        time=times,
        level=pres,
        latitude=(['y','x'],grid.LAT[0,0,:,:].data),
        longitude=(['y','x'],grid.LON[0,0,:,:].data),
    ),
    attrs=dict(
        case='Anqing_202305',
        grid='CN3AH_135X138',
    ),
)
dataset2

<xarray.Dataset>
Dimensions:    (time: 384, level: 38, y: 138, x: 135)
Coordinates:
  * time       (time) datetime64[ns] 2023-05-16 ... 2023-05-31T23:00:00
  * level      (level) float64 1e+03 998.0 995.8 993.4 ... 135.8 101.5 72.89
    latitude   (y, x) float32 29.01 29.01 29.01 29.01 ... 32.7 32.7 32.7 32.7
    longitude  (y, x) float32 114.7 114.7 114.7 114.8 ... 119.0 119.0 119.1
Dimensions without coordinates: y, x
Data variables:
    HADV_NO    (time, level, y, x) float32 dask.array<chunksize=(8, 38, 138, 135), meta=np.ndarray>
    ZADV_NO    (time, level, y, x) float32 dask.array<chunksize=(8, 38, 138, 135), meta=np.ndarray>
    HDIF_NO    (time, level, y, x) float32 dask.array<chunksize=(8, 38, 138, 135), meta=np.ndarray>
    VDIF_NO    (time, level, y, x) float32 dask.array<chunksize=(8, 38, 138, 135), meta=np.ndarray>
    DDEP_NO    (time, level, y, x) float32 dask.array<chunksize=(8, 38, 138, 135), meta=np.ndarray>
    CHEM_NO    (time, level, y, x) float32 dask.array<chunksize=(8, 38, 138, 135), meta=np.ndarray>
    HADV_NO2   (time, level, y, x) float32 dask.array<chunksize=(8, 38, 138, 135), meta=np.ndarray>
    ZADV_NO2   (time, level, y, x) float32 dask.array<chunksize=(8, 38, 138, 135), meta=np.ndarray>
    HDIF_NO2   (time, level, y, x) float32 dask.array<chunksize=(8, 38, 138, 135), meta=np.ndarray>
    VDIF_NO2   (time, level, y, x) float32 dask.array<chunksize=(8, 38, 138, 135), meta=np.ndarray>
    DDEP_NO2   (time, level, y, x) float32 dask.array<chunksize=(8, 38, 138, 135), meta=np.ndarray>
    CHEM_NO2   (time, level, y, x) float32 dask.array<chunksize=(8, 38, 138, 135), meta=np.ndarray>
Attributes:
    case:     Anqing_202305
    grid:     CN3AH_135X138

In [6]:
compression=dict(zlib=True,complevel=5)
encoding={var:compression for var in dataset2.data_vars}
dataset2.to_netcdf('D:/Download/May_PA_NOx-2.nc',encoding=encoding)